In [1]:
# from geopy.geocoders import Nominatim
from geopy import *
import geopy.distance
import pandas as pd
import numpy as np
import math
import itertools
import re
import Levenshtein

In [2]:
excel = r'2020/transmission.xlsx'
sheet = 'Trans.Investments'

df = pd.read_excel(excel, sheet_name=sheet)

# TODO: might not work in later versions, use sanity check.
df.columns = df.iloc[0]
df         = df.drop(index=df.index[0])

# TODO: also contains new substations!

wanted_columns = ['Investment number',
                  'This investment belongs to project number…',
                  'Commissioning Year',
                  'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
                  'Type of Element',
                  'Substation From',
                  'Substation To',
                  'Technology',
                  'Total route length (km)']
status_column = wanted_columns[3]
df[wanted_columns]

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
1,2,1,2024,2,ACTransmissionLine,Pedralva (PT),Sobrado (PT),AC,67
2,4,1,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131
3,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0
4,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30
5,496,4,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21
...,...,...,...,...,...,...,...,...,...
317,1747,1077,2036,1,DCTransmissionLine,Thessaloniki,Dubrovo,DC,110
318,1748,1077,2036,1,DCTransmissionLine,Dubrovo,Ch. Mogila,DC,170
319,1745,1081,2036,1,OffshoreDCTransmissionCable,Tobruk,Arachtos,DC,1070
320,1749,1081,2036,1,DCTransmissionLine,Arachtos,Elbasan,DC,180


In [3]:
wanted    = df[wanted_columns]
# only choose those in permitting or under construction
wanted    = wanted.loc[wanted[status_column].astype(int) >= 3]

ac_lines  = wanted.loc[wanted['Type of Element'] == 'ACTransmissionLine']
dc_lines  = wanted.loc[wanted['Type of Element'] == 'DCTransmissionLine']
on_subst  = wanted.loc[wanted['Type of Element'] == 'OnshoreSubstation']
off_subst = wanted.loc[wanted['Type of Element'] == 'OffshoreSubstation']

In [4]:
# For the time being, remove lines connected to new substations
new_subst = set(on_subst['Substation From']).union(on_subst['Substation To'])
ac_lines  = ac_lines.loc[~ac_lines['Substation From'].isin(new_subst)]
ac_lines  = ac_lines.loc[~ac_lines['Substation To'].isin(new_subst)]

# Use bus names from buses.csv (v0.1.0)
See https://github.com/PyPSA/pypsa-eur/blob/v0.1.0rc/data/entsoegridkit/buses.csv. Data is from 2017 (newer gridkit extracts do not contain 'tags' with substation names).

In [5]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [6]:
no_tags = buses[buses['tags'].isna()].index.size
print(f'{no_tags} buses have no tags.')

yes_tags = buses[~buses['tags'].isna()].index.size
print(f'{yes_tags} buses have tags.')

1238 buses have no tags.
6773 buses have tags.


In [7]:
buses = buses.loc[~buses['tags'].isna()]
buses = buses.loc[buses.symbol == 'Substation']

# Extract 'name_eng' and 'country' from tags in  buses

In [8]:
split_regex = r'("\w+"=>"[^"]*"),' # Form: 'key => value, key => value, ...'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"' # Form: 'key => value'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [9]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [10]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

### List of all duplicates

In [11]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]

for name in duplicated.name.unique():
    print(name)
    for index, row in curated_buses.loc[curated_buses['name'] == name].iterrows():
        print(f"({row['x']}, {row['y']}), {row['country']}")
    print('----')

Jeddah
(39.716949, 21.985075), SA
(39.711456, 22.038549), SA
----
Al Madina
(39.635925, 24.775513), SA
(39.508209, 24.718143), SA
----
I. Baroud
(30.734253, 30.955236), EG
(30.890808, 30.855079), EG
----
Local
(25.041962, 31.598422), LY
(24.134216, 31.960318), LY
----
Sahab
(36.021423, 31.809895), JO
(36.478729, 32.261588), JO
----
Tazoult
(6.378937, 35.424868), DZ
(6.49154699999999, 35.373375), DZ
----
Saida
(35.400696, 33.587167), LB
(0.146942000000011, 34.908458), DZ
----
Tizi Ouzou
(4.15557899999999, 36.698154), DZ
(4.000397, 36.561497), DZ
----
Jerada
(-2.06405600000001, 33.876117), MA
(-2.11898799999999, 34.335498), MA
----
Taza
(-4.139099, 34.271971), MA
(-3.95919799999999, 34.268566), MA
----
Naama 400
(-0.365294999999999, 33.26625), DZ
(-0.429840000000002, 33.366091), DZ
----
Brindisi /S.
(17.992859, 40.571197), IT
(17.862396, 40.520063), IT
----
Tunis
(10.140381, 36.73118), TN
(10.055237, 36.79499), TN
----
PB 1
(7.04772899999999, 36.857648), DZ
(7.138367, 36.891703), DZ
----

### Same name and country, large deviations

In [12]:
curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']

,name,country,x,y
2720,Yuzhnaya,RU,44.817352,48.155093
3851,Yuzhnaya,RU,50.674438,52.002638
3905,Yuzhnaya,RU,36.268616,51.642737
3927,Yuzhnaya,RU,38.685608,51.843414
5378,Yuzhnaya,RU,59.824677,56.576128


### Same name, different country, large deviation

In [13]:
curated_buses.loc[curated_buses.name == 'Saida']

,name,country,x,y
617,Saida,LB,35.400696,33.587167
833,Saida,DZ,0.146942,34.908458


In [14]:
curated_buses.loc[curated_buses.name == 'Titan']

,name,country,x,y
1986,Titan,AL,19.786377,41.619549
2825,Titan,UA,33.767853,46.195993
5825,Titan,RU,34.026031,67.451763


## (TODO) Add new substations

In [15]:
on_subst

# extract country if it matches regex
# otherwise, np.NAN

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
3,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0
7,499,4,2022,3,OnshoreSubstation,Beariz (ES),Beariz (ES),AC,0
8,500,4,2022,3,OnshoreSubstation,Ponte de Lima (PT),Ponte de Lima (PT),AC,0
59,715,138,2023,4,OnshoreSubstation,Stalpu (RO),Stalpu (RO),AC,0
68,701,144,2025,4,OnshoreSubstation,Resita (RO),Resita (RO),AC,0
69,705,144,2025,3,OnshoreSubstation,Timisoara (RO),Timisoara (RO),AC,0
110,1711,200,2023,4,OnshoreSubstation,Kocin (CZ),NaN,AC,0
111,1712,200,2021,4,OnshoreSubstation,Vitkov (CZ),NaN,AC,0
123,631,227,2024,3,OnshoreSubstation,Bajina Basta (RS),Bajina Basta (RS),AC,0
126,1528,227,2020,4,OnshoreSubstation,SS Kraljevo,SS Kraljevo,AC,0


## Remove '(\<Country Code\>) ' from tyndp substation name strings, add new column instead
Otherwise, this could negatively impact the Levenshtein distance.

In [16]:
subst_regex   = r'(?P<place>.+)\s?[\[(](?P<country>\w{2})[)\]]' # Form: 'Glorenza (IT)'
subst_pattern = re.compile(subst_regex)

# use this if other pattern does not match to remove comments in parentheses
# e.g. 'Molai (through Sklavouna Terminal)'
alt_regex   = r'(?P<place>.+)\s?[\[(].*[)\]]'
alt_pattern = re.compile(alt_regex)

fr_names     = []
fr_countries = []
to_names     = []
to_countries = []

for index, row in ac_lines.iterrows():    
    fr = row['Substation From']
    to = row['Substation To']
    
    # default values if regex does not match
    fr_name = fr
    to_name = to    
    fr_country = np.NAN
    to_country = np.NAN
    
    fr_match = subst_pattern.match(fr)
    to_match = subst_pattern.match(to)
    
    if fr_match:
        fr_name    = fr_match.group('place').strip()
        fr_country = fr_match.group('country').strip()
    else:
        fr_alt_match = alt_pattern.match(fr)
        if fr_alt_match:
            fr_name = fr_alt_match.group('place')
        
    if to_match:
        to_name    = to_match.group('place').strip()
        to_country = to_match.group('country').strip()
    else:        
        to_alt_match = alt_pattern.match(to)
        if to_alt_match:
            to_name = to_alt_match.group('place')
        
    fr_names.append(fr_name)
    fr_countries.append(fr_country)
    to_names.append(to_name)
    to_countries.append(to_country)

In [17]:
ac_lines['Substation From'] = fr_names
ac_lines['Substation To'] = to_names
ac_lines['Country1'] = fr_countries
ac_lines['Country2'] = to_countries
ac_lines

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),Country1,Country2
2,4,1,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131,NaN,NaN
5,496,4,2022,3,ACTransmissionLine,Fontefria,Vila Nova de Famalicão,AC,140.21,ES,PT
10,60,23,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80,FR,BE
11,614,26,2023,4,ACTransmissionLine,Nauders,Glorenza,AC,26,AT,IT
14,90,33,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80,IT,IT
...,...,...,...,...,...,...,...,...,...,...,...
286,1689,1046,2029,3,ACTransmissionLine,Tuomela B,Nuojuankangas,AC,100,NaN,NaN
295,1716,1055,2020,4,ACTransmissionLine,Chania I,Molai,AC,179,NaN,NaN
304,645,1059,2027,3,ACTransmissionLine,Laino,Altomonte,AC,50,IT,IT
305,1727,1059,2030,3,ACTransmissionLine,Montecorvino,Benevento,AC,0,IT,IT


## create mapping from all unique tyndp substation names to substation names from 'buses'

In [18]:
tyndp_subs   = set(ac_lines['Substation From']).union(set(ac_lines['Substation To']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, Levenshtein.distance(bus.lower(), tyndp.lower())) for bus in buses_subs], key=lambda t: t[1])[0]
    
    tyndp_to_bus[tyndp] = closest

In [19]:
tyndp_to_bus

{'Ganderkesee': 'Ganderkesee',
 'Tsirguliina ': 'Tsirguliina',
 'Hinkley Point': 'Hinkley Point',
 'Villanova': 'Villanova',
 'Stanislawow': 'Stanisławów',
 'Balti': 'Salt',
 'Kassoe': 'Kassø',
 'Tuomela B': 'Tudela',
 'Geertruidenberg': 'Geertruidenberg',
 'VRTOJBA': 'Taba',
 'Sajoivanka': 'Sajóivánka',
 'Bitola': 'Bitola',
 'Gutinas': 'Gutinaş',
 'Krimpen': 'Krimpen',
 'Petajavesi': 'Petäjävesi',
 'Bassecourt': 'Bassecourt',
 'Valmiera': 'Valmiera',
 'Krajnik': 'Krajnik',
 'Zaya': 'Zarya',
 'Vila Nova de Famalicão': 'La Roda de Andalucia',
 'Woodland': 'Woodland',
 'Rilland': 'Rifland',
 'Molai ': 'Mora',
 'Chania I': 'Chantada',
 'Dollern': 'Dollern',
 'Wilhelmshaven': 'Wilhelmshaven',
 'Eindhoven': 'Eindhoven',
 'Fontefria': 'Fontefria',
 'Ekhyddan': 'Ekudden',
 'Deliceto': 'Lieto',
 'Pyhänselkä': 'Pyhänselkä',
 'Cirkovce': 'Cirkovce',
 'SS Lastva': 'Shalati',
 'Varna': 'Varna',
 'Keminmaa': 'Keminmaa',
 'Beznau': 'Beznau',
 'Prati di Vizze': 'La Riviere',
 'Heviz (HU) \\ Zerjavin

### Helper functions: Out of all possible pairs of locations from two lists, get the pair whose distance is closest to the specified (line) length
Deals with problem of multiple places in same country sharing a name.

In [20]:
def extract_coords(rows):
    coordinates = []
    for index, row in rows.iterrows():
        coordinates.append((row['x'], row['y']))
    return coordinates

In [21]:
def match_pair_with_length(s1_rows, s2_rows, length):
    s1_coords = extract_coords(s1_rows)
    s2_coords = extract_coords(s2_rows)
    
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(a,b).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

In [22]:
# Example
buses_s1 = curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']
buses_s2 = curated_buses.loc[curated_buses['name'] == 'Liteynaya']

best = match_pair_with_length(buses_s1, buses_s2, 430)
best

((38.685608, 51.843414), (35.021667, 53.707275), 439.2404656986871)

# Match start- and endpoints of lines to substations from buses.csv

In [24]:
fr_to_tuples = {}
error_tuples = {}

for index, row in ac_lines.iterrows():    
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    fr = row['Substation From']
    to = row['Substation To']
    
    fr_country = row['Country1']
    to_country = row['Country2']
            
    s1 = tyndp_to_bus[fr]
    s2 = tyndp_to_bus[to]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1 = curated_buses.loc[curated_buses.name == s1]
    buses_s2 = curated_buses.loc[curated_buses.name == s2]
    
    # If we were able to extract country from name, restrict chosen rows to this country.
    if not pd.isna(fr_country):
        buses_s1 = buses_s1.loc[buses_s1['country'] == fr_country]
    if not pd.isna(to_country):
        buses_s2 = buses_s2.loc[buses_s2['country'] == to_country]
    
    # TODO: use at least one!  
    if buses_s1.empty or buses_s2.empty:
        # print(f'{fr}, {fr_country} <-> {to}, {to_country}')
        tpl = (s1, np.NAN, np.NAN, s2, np.NAN, np.NAN, np.NAN)
        error_tuples[index] = tpl
        continue
    
    # Choose pair which matches length best
    length = row['Total route length (km)']
    (x1, y1), (x2, y2), coord_dist = match_pair_with_length(buses_s1, buses_s2, length)
        
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)
    
    # TODO: how to choose an appropriate tolerance?
    if not math.isclose(coord_dist, length, rel_tol=0.45):
        error_tuples[index] = tpl
    else:
        fr_to_tuples[index] = tpl

Fontefria, ES <-> Vila Nova de Famalicão, PT
Nauders, AT <-> Glorenza, IT
Gabcikovo, SK <-> Gonyu, HU
Woodland, IE <-> Turleenan, NI
Ostroleka, PL <-> Stanislawow, PL
Ekhyddan, SE <-> Nybro/Hemsjo, SE
Deliceto, IT <-> Bisaccia, IT
Maritsa East 1, BG <-> N.Santa, GR
Maritsa East 1, BG <-> Plovdiv, BG
Maritsa East 1, BG <-> Maritsa East 3, BG
Maritsa East 1, BG <-> Burgas, BG
Klixbüll, DE <-> Endrup , nan
Brunsbüttel, DE <-> Klixbüll, DE
Liefkenshoek, BE <-> Mercator, BE
Wullenstetten, DE <-> Austrian National border, AT
Prati di Vizze, IT <-> Steinach, AT


In [26]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

result = ac_lines.copy()
result = result.join(coordinates)

percentage = coordinates.index.size / ac_lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

print('Lines where we probably found the correct coordinates:')
result.loc[~result.s1.isna()]

51.85185185185185% of lines are probably correct.
Lines where we probably found the correct coordinates:


,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),Country1,Country2,s1,x1,y1,s2,x2,y2,coord_dist
2,4,1,2022,3,ACTransmissionLine,V.Minho,Feira,AC,131,NaN,NaN,V. Minho,-7.761841,41.582580,Feira,-8.368835,40.953974,96.475112
10,60,23,2022,4,ACTransmissionLine,Avelin/Mastaing,Horta,AC,80,FR,BE,Mastaing,3.353577,50.316531,Horta,3.574677,51.015483,81.423032
14,90,33,2023,3,ACTransmissionLine,Calenzano,Colunga,AC,80,IT,IT,Calenzano,11.126404,43.871168,Colunga,11.284332,44.494546,70.285438
20,144,39,2020,4,ACTransmissionLine,Audorf,Kassoe,AC,110,DE,DK,Audorf,9.759979,54.305307,Kassø,9.295807,55.074436,98.827047
26,1501,48,2020,4,ACTransmissionLine,Rimavska Sobota,Sajoivanka,AC,48,SK,HU,Rimavská Sobota,19.958038,48.379970,Sajóivánka,20.560913,48.263084,67.850755
27,386,62,2020,4,ACTransmissionLine,Kilingi-Nomme,Riga CHP2,AC,205,EE,LV,Kilingi-Nomme,25.166931,58.185909,Riga HES,23.832092,56.830944,201.790950
28,735,62,2021,4,ACTransmissionLine,Harku,Sindi,AC,175,EE,EE,Harku,24.378662,59.396171,Sindi,24.689026,58.377238,108.818618
30,458,78,2024,3,ACTransmissionLine,Hinkley Point,Seabank,AC,60,GB,GB,Hinkley Point,-3.210754,51.135416,Seabank,-2.633972,51.533523,77.631605
40,1488,103,2022,4,ACTransmissionLine,Diemen,Ens,AC,75,NaN,NaN,Diemen,5.067444,52.238733,Ens,5.766449,52.653062,89.908740
41,1490,103,2024,3,ACTransmissionLine,Ens,Zwolle,AC,32,NaN,NaN,Ens,5.766449,52.653062,Zwolle,6.212769,52.511206,51.795486


In [27]:
error_lines = result.loc[result.s1.isna()]
error_subst = set(error_lines['Substation From']).union(error_lines['Substation To'])

print('')
{(k,tyndp_to_bus[k]) for k in error_subst}

{('Austrian National border', 'Craiova Nord'),
 ('Balti', 'Salt'),
 ('Benevento', 'Benevento'),
 ('Bickigen', 'Bickigen'),
 ('Bisaccia', 'Bisaccia'),
 ('Bitenai', 'Bitėnai'),
 ('Bitola', 'Bitola'),
 ('Borssele', 'Borssele'),
 ('Brunsb\x81üttel', 'Brunsbüttel'),
 ('Burgas', 'Burgas'),
 ('Chamoson', 'Chamoson'),
 ('Chania I', 'Chantada'),
 ('Chippis', 'Chiari'),
 ('DEKANI', 'Swani'),
 ('Darbenai', 'Darna'),
 ('Deliceto', 'Lieto'),
 ('Ekhyddan', 'Ekudden'),
 ('Elbasan', 'Elbistan'),
 ('Endrup ', 'Endrup'),
 ('Fontefria', 'Fontefria'),
 ('Gabcikovo', 'Gabčíkovo'),
 ('Glorenza', 'Glorenza'),
 ('Gonyu', 'Konya'),
 ('KHAE', 'Ghat'),
 ('Keminmaa ', 'Keminmaa'),
 ('Klixbüll', 'Kitwell'),
 ('La Punt', 'La Font'),
 ('Liefkenshoek', 'Lindenholz'),
 ('Lietuvos E', 'Lieto'),
 ('Lillo - Liefkenshoek', 'Los Vientos'),
 ('Maritsa East 1', 'Misrata East 1'),
 ('Maritsa East 3', 'Misrata East 1'),
 ('Mercator', 'Mercator'),
 ('Messaure ', 'Missour'),
 ('Mettlen', 'Mettlen'),
 ('Molai ', 'Mora'),
 ('Monte

# Determine coordinates using geopy

In [29]:
locator = Nominatim(user_agent='esm_group')
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.05)

NameError: name 'RateLimiter' is not defined

In [ ]:
x1 = []
y1 = []
x2 = []
y2 = []

error_rows = []

for index, row in ac_lines.iterrows():
    fr   = row['Substation From']
    to   = row['Substation To']
    dist = row['Total route length (km)']

    fr_loc = geocode(fr)
    to_loc = geocode(to)

    if fr_loc is None or to_loc is None:
        error_rows.append([row.values])
        continue
        
    fr_coords  = fr_loc.latitude, fr_loc.longitude
    to_coords  = to_loc.latitude, to_loc.longitude
    coord_dist = distance.distance(fr_coords, to_coords).km

    if math.isclose(coord_dist, dist, rel_tol=0.25):
        x1.append(fr_coords[0])
        y1.append(fr_coords[1])
        
        x2.append(to_coords[0])
        y2.append(to_coords[1])
    else:
        error_rows.append([row.values])
        # x1.append(np.NAN)
        # y1.append(np.NAN)
        
        # x2.append(np.NAN)
        # y2.append(np.NAN)

In [ ]:
len(x1), len(error_rows)

In [ ]:
error_rows